### Data Cleaning/Wrangling

In [1]:
import pandas as pd
import numpy as np

First, I'll create a dataframe from the dataset, which is in three CSV files due to GitHub's file size limit. I specify latin1 encoding because there was at least one byte that could not be interpreted using utf-8. (Some) missing values are encoded with "__NA__" in this dataset.

In [2]:
df1 = pd.read_csv('CCES2016.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])
df2 = pd.read_csv('CCES2.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])
df3 = pd.read_csv('CCES3.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])

#'utf-8' codec can't decode byte 0xe1 in position 5: unexpected end of data

df = df1.append(df2)
df = df.append(df3)

df = df.reset_index()

C:\Users\joshy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (50,51,52,53,54,55,79,119,254,262,270,271,274,286,296,298,300,302,304,306,308,310,314,316,318,322,326,328,332,333,370,410,411,412,413,414,415,452,458,460,462,494,535) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\joshy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (46,47,48,49,50,51,52,53,54,55,79,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,164,165,166,167,168,169,170,199,200,201,203,228,239,240,242,243,254,262,264,270,271,274,278,279,286,296,298,300,302,304,306,308,310,312,314,316,317,318,320,322,324,326,328,330,332,334,370,406,407,408,409,410,411,412,413,414,415,443,450,452,458,460,520,535) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=resu

In [3]:
df.head()

,V101,commonweight_vv,commonweight_vv_post,commonweight_vv_lgbt,commonweight,commonweight_post,tookpost,lookupzip,inputstate,cdid113,...,CL_E2016PPEP,CL_E2016PPVM,CL_E2016PEP,CL_E2016PVM,starttime_pre,endtime_pre,starttime_post,endtime_post,dma,dmaname
0,222168628,1.344214,1.042267,1.195346,0.944967,0.730450,Yes,3442,New Hampshire,2,...,NaN,NaN,NaN,NaN,05:44.9,36:38.8,24:32.9,40:18.8,NaN,NaN
1,273691199,1.182352,1.280103,1.280565,0.767256,0.892838,Yes,70118,Louisiana,2,...,NaN,NaN,NaN,NaN,54:58.5,20:41.7,02:55.1,09:26.9,622.0,NEW ORLEANS
2,284214415,0.217396,NaN,0.248930,0.090604,0.000000,No,63851,Missouri,8,...,NaN,NaN,NaN,NaN,54:12.8,24:44.8,NaN,NaN,632.0,PADUCAH-CAPE GIRARD-HARSBG
3,287557695,0.532320,NaN,0.928750,0.142583,0.000000,No,36703,Alabama,7,...,NaN,NaN,NaN,NaN,06:21.3,26:00.1,NaN,NaN,698.0,MONTGOMERY-SELMA
4,290387662,1.261106,1.092519,1.161894,1.206498,1.019007,Yes,80526,Colorado,2,...,NaN,NaN,NaN,NaN,56:31.1,14:12.2,09:28.0,21:05.9,751.0,DENVER


The birth year ("birthyr") column is missing in the CSV file containing the 2016 CCES data, for some reason. Fortunately, I was able to find this column in another spreadsheet that showed the cumulative CCES results from 2006 to 2017, which uses the same case ID index. The cumuative16.csv file contains the rows from that spreadsheet for the 2016 survey.

In [4]:
cumulative = pd.read_csv('cumulative16.csv', encoding = 'latin1')
cumulative = cumulative[['case_id', 'birthyr']]
cumulative.head()

,case_id,birthyr
0,222168628,1969
1,273691199,1994
2,284214415,1964
3,287557695,1988
4,290387662,1982


Now, I merge the two dataframes, using the names of the case ID column for each dataframe.

In [5]:
df = pd.merge(df, cumulative, left_on = 'V101', right_on = 'case_id')

In [6]:
df['birthyr'] = df['birthyr_y']

In [7]:
df.drop(['birthyr_y'], axis = 1)
df[['V101', 'birthyr']].head()

,V101,birthyr
0,222168628,1969
1,273691199,1994
2,284214415,1964
3,287557695,1988
4,290387662,1982


In [8]:
#ensure that case IDs and birth years match for both dataframes
different_index = []
for i in range(64600):
    if df['V101'].iloc[i] != cumulative['case_id'].iloc[i]:
        different_index.append(i)
        
different_year = []
for i in range(64600):
    if df['birthyr'].iloc[i] != cumulative['birthyr'].iloc[i]:
        different_year.append((i, df['birthyr'].iloc[i], cumulative['birthyr'].iloc[i] ))
        
print(len(different_index))
print(len(different_year))

0
0


### Column selection and Missing Values

Now, I have the data imported into a dataframe. There are a large number of columms, many of which are not likely to be useful for this project. First, I'll print an alphabetical list of columns:

In [9]:
columns_indices = []

for i in range(len(df.columns)):
    columns_indices.append((i, df.columns[i]))
    
columns_indices = sorted(columns_indices, key = lambda x: x[1].lower())

for item in columns_indices:
    if item[0] < 10:
        print(str(item[0]) + "   " + str(item[1]))
    elif item[0] < 100:
        print(str(item[0]) + "  " + str(item[1]))
    else:
        print(str(item[0]) + " " + str(item[1]))

70  add_confirm
104 Asian_origin_1
113 Asian_origin_10
114 Asian_origin_11
115 Asian_origin_12
116 Asian_origin_13
117 Asian_origin_14
118 Asian_origin_15
105 Asian_origin_2
106 Asian_origin_3
107 Asian_origin_4
108 Asian_origin_5
109 Asian_origin_6
110 Asian_origin_7
111 Asian_origin_8
112 Asian_origin_9
119 Asian_origin_t
372 AskParty_post
565 birthyr
75  birthyr_x
564 birthyr_y
563 case_id
121 CC16_300_1
122 CC16_300_2
123 CC16_300_3
124 CC16_300_4
125 CC16_300_5
126 CC16_300_6
127 CC16_300b
128 CC16_300c
129 CC16_300d_1
130 CC16_300d_2
131 CC16_300d_3
132 CC16_300d_4
133 CC16_300d_5
134 CC16_301a
135 CC16_301b
136 CC16_301c
137 CC16_301d
138 CC16_301e
139 CC16_301f
140 CC16_301g
141 CC16_301h
142 CC16_301i
143 CC16_301j
144 CC16_301k
145 CC16_301l
146 CC16_301m
147 CC16_301n
148 CC16_301o
149 CC16_302
150 CC16_303
151 CC16_304
152 CC16_305_1
161 CC16_305_10
162 CC16_305_11
153 CC16_305_2
154 CC16_305_3
155 CC16_305_4
156 CC16_305_5
157 CC16_305_6
158 CC16_305_7
159 CC16_305_8
160 C

Next, I look at how many missing values there are by column:

In [10]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1])

for i in range(1, len(nan_counts)):
    print(nan_counts[i][0], nan_counts[i][1])

commonweight_vv 0
commonweight 0
commonweight_post 0
tookpost 0
lookupzip 0
inputstate 0
cdid113 0
cdid115 0
GovCand1Incumbent 0
GovCand2Incumbent 0
HouseCand1Incumbent 0
HouseCand2Incumbent 0
SenCand1Incumbent 0
SenCand2Incumbent 0
CCEStake 0
comptype 0
votereg 0
gender 0
educ 0
race 0
multrace_1 0
multrace_2 0
multrace_3 0
multrace_4 0
multrace_5 0
multrace_8 0
multrace_98 0
ccesmodule 0
CC16_300_1 0
CC16_300_2 0
CC16_300_3 0
CC16_300_4 0
CC16_300_5 0
CC16_300_6 0
CC16_305_1 0
CC16_305_2 0
CC16_305_3 0
CC16_305_4 0
CC16_305_5 0
CC16_305_6 0
CC16_305_7 0
CC16_305_8 0
CC16_305_9 0
CC16_305_10 0
CC16_305_11 0
CC16_331_1 0
CC16_331_2 0
CC16_331_3 0
CC16_331_7 0
CC16_331_9 0
employ 0
milstat_1 0
milstat_2 0
milstat_3 0
milstat_4 0
milstat_5 0
healthins_1 0
healthins_2 0
healthins_3 0
healthins_4 0
healthins_5 0
healthins_6 0
CL_matched 0
starttime_pre 0
endtime_pre 0
case_id 0
birthyr_y 0
birthyr 0
CC16_351K 19
pid3 20
faminc 21
CC16_320a 22
CC16_320c 31
CC16_334a 32
CC16_320b 33
CC16_334

215 variables have fewer than 10% of the data missing.

In [11]:
df.groupby(['tookpost']).count()

,V101,commonweight_vv,commonweight_vv_post,commonweight_vv_lgbt,commonweight,commonweight_post,lookupzip,inputstate,cdid113,cdid115,...,CL_E2016PVM,starttime_pre,endtime_pre,starttime_post,endtime_post,dma,dmaname,case_id,birthyr_y,birthyr
tookpost,,,,,,,,,,,,,,,,,,,,,
No,11701,11701,43,4280,11701,11701,11701,11701,11701,11701,...,576,11701,11701,0,0,10827,10827,11701,11701,11701
Yes,52899,52899,52899,52868,52899,52899,52899,52899,52899,52899,...,7742,52899,52899,52899,52899,48706,48706,52899,52899,52899


52899 took the post survey.

Because the number of variables was so large, I created a spreadsheet to keep track of notes for each variable and decide whether it should be included in this project. Here I import and extract data from this spreadsheet.

In [12]:
vars_df = pd.read_csv('variable_list.csv', encoding = 'latin1')
variables = vars_df.loc[vars_df['Include'] == 'yes']['Name'].tolist()

In [13]:
print(variables)

['tookpost', 'birthyr', 'gender', 'educ', 'race', 'CC16_331_1', 'CC16_331_2', 'CC16_331_3', 'CC16_331_7', 'employ', 'milstat_1', 'milstat_2', 'milstat_3', 'milstat_4', 'CC16_351K', 'faminc', 'CC16_334a', 'CC16_334b', 'pew_religimp', 'ideo5', 'CC16_334d', 'pew_bornagain', 'CC16_334c', 'CC16_333d', 'marstat', 'CC16_333b', 'CC16_333a', 'CC16_332a', 'CC16_332d', 'CC16_332e', 'CC16_333c', 'religpew', 'ownhome', 'CC16_332c', 'CC16_351I', 'CC16_332b', 'child18', 'CC16_351B', 'CC16_351E', 'union', 'CC16_327', 'immstat', 'CC16_351G', 'CC16_351F', 'CC16_330a', 'CC16_351H', 'CC16_330e', 'CC16_330b', 'unionhh', 'CC16_330d', 'CC16_335', 'hispanic', 'sexuality', 'investor', 'trans', 'inputstate_post', 'votereg_post', 'CC16_414_1', 'CC16_414_2', 'CC16_414_3', 'CC16_414_4', 'CC16_414_5', 'CC16_414_6', 'CC16_417a_1', 'CC16_417a_2', 'CC16_417a_3', 'CC16_417a_4', 'CC16_417a_5', 'CC16_421a', 'CC16_422d', 'CC16_422c', 'edloan', 'CC16_422f', 'CC16_422e', 'CC16_426_1', 'CC16_426_2', 'CC16_426_3', 'CC16_426_5

In [14]:
df = df[variables]

I also renamed variables to make things easier. The new names also come from the same spreadsheet:

In [15]:
newnames = {}
for row in vars_df[['Name', 'Rename']].itertuples():
    if row[1] in variables:
        #print(row[1])
        newnames[row[1]] = row[2]
        
df = df.rename(index = str, columns = newnames)

In [16]:
for key, value in newnames.items():
    print(key,value)

tookpost tookpost
birthyr birthyr
gender gender
educ educ
race race
CC16_331_1 legalstatus
CC16_331_2 borderpatrol
CC16_331_3 legalstatusHS
CC16_331_7 deport
employ employ
milstat_1 milstat_1
milstat_2 milstat_2
milstat_3 milstat_3
milstat_4 milstat_4
CC16_351K minwage12
faminc faminc
CC16_334a mandatorymin
CC16_334b bodycamera
pew_religimp pew_religimp
ideo5 ideo5
CC16_334d threestrikes
pew_bornagain pew_bornagain
CC16_334c increasepolice
CC16_333d cleanair
marstat marstat
CC16_333b fuelefficiency
CC16_333a EPACO2
CC16_332a abortionchoice
CC16_332d abortioncoverage
CC16_332e federalabortion
CC16_333c renewables
religpew religpew
ownhome ownhome
CC16_332c abortion20wks
CC16_351I repealACA
CC16_332b banmostabortion
child18 child18
CC16_351B TPP
CC16_351E NCLB
union union
CC16_327 primary
immstat immstat
CC16_351G Iransanctions
CC16_351F infraspending
CC16_330a backgroundcheck
CC16_351H medicarereform
CC16_330e concealedcarry
CC16_330b gunregistry
unionhh unionhh
CC16_330d banassault
CC1

In [17]:
for col in df.columns:
    print(col)

tookpost
birthyr
gender
educ
race
legalstatus
borderpatrol
legalstatusHS
deport
employ
milstat_1
milstat_2
milstat_3
milstat_4
minwage12
faminc
mandatorymin
bodycamera
pew_religimp
ideo5
threestrikes
pew_bornagain
increasepolice
cleanair
marstat
fuelefficiency
EPACO2
abortionchoice
abortioncoverage
federalabortion
renewables
religpew
ownhome
abortion20wks
repealACA
banmostabortion
child18
TPP
NCLB
union
primary
immstat
Iransanctions
infraspending
backgroundcheck
medicarereform
concealedcarry
gunregistry
unionhh
banassault
gaymarriage
hispanic
sexuality
investor
trans
inputstate_post
votereg_post
militaryoil
militaryterror
militarycivilwar
militarydemocracy
militaryally
militaryUN
polmeeting
polsign
campaignwork
campaigndonate
donateblood
partyID
whiteadvantage
angryracism
edloan
racismrare
fearrace
statewelfare
statehealthcare
stateedu
stateinfra
runoffice
stateLE
voted2016
2012vote
presvote


I am interested in people who took the post-election survey, indicating how they voted.

In [18]:
df = df[df['tookpost'] == 'Yes']

#df.info()

Now, let's look at how many missing values are remaining after removing most of the columns:

In [19]:
df.groupby(['2012vote']).size()

2012vote
Barack Obama    23395
Did not vote       69
Don't recall      627
Mitt Romney     16714
Someone else     1371
dtype: int64

In [20]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse=True)

for i in range(0, len(nan_counts)):
    print(nan_counts[i])

('2012vote', 10723)
('presvote', 7657)
('hispanic', 3903)
('voted2016', 3814)
('gaymarriage', 431)
('banassault', 355)
('gunregistry', 325)
('concealedcarry', 311)
('unionhh', 306)
('medicarereform', 232)
('stateLE', 207)
('Iransanctions', 188)
('backgroundcheck', 186)
('runoffice', 176)
('stateinfra', 172)
('infraspending', 168)
('stateedu', 167)
('statehealthcare', 145)
('statewelfare', 139)
('immstat', 127)
('fearrace', 124)
('racismrare', 121)
('investor', 119)
('NCLB', 118)
('TPP', 111)
('edloan', 110)
('primary', 108)
('child18', 96)
('union', 87)
('repealACA', 82)
('banmostabortion', 78)
('abortion20wks', 66)
('angryracism', 65)
('trans', 62)
('whiteadvantage', 62)
('ownhome', 61)
('renewables', 55)
('abortionchoice', 53)
('federalabortion', 53)
('EPACO2', 52)
('increasepolice', 49)
('fuelefficiency', 48)
('abortioncoverage', 48)
('religpew', 46)
('cleanair', 44)
('threestrikes', 39)
('marstat', 39)
('sexuality', 31)
('bodycamera', 30)
('pew_bornagain', 29)
('mandatorymin', 28)


2012vote (who did you vote for in 2012) had the most missing data. However, it still provides interesting information, so I'll convert some of the values to dummies. 

In [21]:
df['Obama'] = (df['2012vote'] == 'Barack Obama').astype(int)
df['Romney'] = (df['2012vote'] == 'Mitt Romney').astype(int)

In [22]:
df[['Obama','Romney', '2012vote']].head(20)

,Obama,Romney,2012vote
0,0,0,NaN
1,0,0,NaN
4,1,0,Barack Obama
5,1,0,Barack Obama
6,0,1,Mitt Romney
8,0,1,Mitt Romney
9,0,1,Mitt Romney
10,1,0,Barack Obama
11,0,0,Someone else
12,0,0,NaN


In [23]:
df = df.drop(['2012vote'], axis = 1)

The variable with the next-largest number of missing rows is presvote (Who did you vote for in 2016?). This is an essential outcome variable so I cannot exclude it. Fortunately, the missing rows come from respondents who did not answer this question because they already indicated that they did not vote in another question.

In [24]:
df.groupby(['presvote']).size()

presvote
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election       81
I'm not sure                        229
Jill Stein (Green)                  913
Other                              1136
dtype: int64

I create a column encoding whether the respondent voted:

In [25]:
def voted(s):
    if "definitely voted" in str(s):
        return 1
    else:
        return 0

df['voted'] = df['voted2016'].apply(voted)

In [26]:
df.groupby(['voted']).size()

voted
0     7607
1    45292
dtype: int64

In [27]:
df.groupby(['presvote']).size()

presvote
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election       81
I'm not sure                        229
Jill Stein (Green)                  913
Other                              1136
dtype: int64

Now I populate the CC16_410a column based on the voted column:

In [28]:
df.loc[df['voted'] == 0, 'presvote'] = 'I didn\'t vote in this election'

In [29]:
df.groupby(['presvote']).size()

presvote
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election     7688
I'm not sure                        229
Jill Stein (Green)                  913
Other                              1136
dtype: int64

In [30]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse=True)

for i in range(0, 20):
    print(nan_counts[i])

('hispanic', 3903)
('voted2016', 3814)
('gaymarriage', 431)
('banassault', 355)
('gunregistry', 325)
('concealedcarry', 311)
('unionhh', 306)
('medicarereform', 232)
('stateLE', 207)
('Iransanctions', 188)
('backgroundcheck', 186)
('runoffice', 176)
('stateinfra', 172)
('infraspending', 168)
('stateedu', 167)
('statehealthcare', 145)
('statewelfare', 139)
('immstat', 127)
('fearrace', 124)
('racismrare', 121)


Presidential vote is now almost entirely filled out. vote2016 itself has a lot of missing data due to respondents not being asked that question, but the information is contained in presvote anyway, so I won't worry about it.

In [31]:
df = df.drop(['voted2016'], axis = 1)

In [32]:
df = df.drop(['voted'], axis = 1)

The next column is the hispanic column. Again, the missing values come from respondents not being asked the question, and I can get this data from the race column (if respondents already answered "Hispanic" when asked their race, they were not asked again whether they were Hispanic). 

In [33]:
df.groupby(['race']).size()

race
Asian               1739
Black               4931
Hispanic            3742
Middle Eastern        99
Mixed               1169
Native American      440
Other                680
White              40099
dtype: int64

In [34]:
df.groupby(['hispanic']).size()

hispanic
No     47389
Yes     1607
dtype: int64

In [35]:
df.loc[df['race'] == 'Hispanic', 'hispanic'] = 'Yes'

In [36]:
df.groupby(['hispanic']).size()

hispanic
No     47389
Yes     5349
dtype: int64

In [37]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse = True)

for i in range(0, 10):
    print(nan_counts[i])

('gaymarriage', 431)
('banassault', 355)
('gunregistry', 325)
('concealedcarry', 311)
('unionhh', 306)
('medicarereform', 232)
('stateLE', 207)
('Iransanctions', 188)
('backgroundcheck', 186)
('runoffice', 176)


For the other columns, missing values come from respondents skipping the question, and the number of missing values is relatively small.

### Implicit missing values

However, there are still missing values that are not coded as NaN yet, because some answers represent the respondent declining to provide any information. I will now print the unique values for each column to check for this:

In [38]:
for col in df.columns.sort_values():
    print (col, df[col].unique())

EPACO2 ['Oppose' 'Support' nan]
Iransanctions ['For' 'Against' nan]
NCLB ['For' 'Against' nan]
Obama [0 1]
Romney [0 1]
TPP ['For' 'Against' nan]
abortion20wks ['Support' 'Oppose' nan]
abortionchoice ['Support' 'Oppose' nan]
abortioncoverage ['Oppose' 'Support' nan]
angryracism ['Somewhat agree' 'Strongly agree' 'Neither agree nor disagree'
 'Somewhat disagree' nan 'Strongly disagree']
backgroundcheck ['Support' 'Oppose' nan]
banassault ['Oppose' 'Support' nan]
banmostabortion ['Oppose' 'Support' nan]
birthyr [1969 1994 1982 1963 1962 1957 1946 1985 1976 1988 1979 1961 1958 1945 1990
 1971 1940 1956 1981 1960 1978 1959 1967 1954 1984 1952 1964 1991 1965 1942
 1949 1939 1934 1944 1975 1977 1966 1948 1970 1992 1953 1980 1974 1955 1983
 1986 1937 1936 1951 1947 1987 1997 1950 1941 1938 1931 1989 1968 1973 1996
 1995 1993 1935 1943 1932 1927 1933 1972 1923 1925 1930 1929 1928 1924 1998
 1921 1926 1922]
bodycamera ['Support' 'Oppose' nan]
borderpatrol ['Yes' 'No']
campaigndonate ['No' 'Yes'

Family income (faminc), sexuality, trans have a "prefer not to say" option. Ideology (ideo5), presidential vote (CC16_410a), has a "Not sure" option, which is arguably missing information. Voter registration (votereg_post) has a "don't know" option.

In [39]:
df.groupby(['ideo5']).size()

ideo5
Conservative         12183
Liberal              10147
Moderate             17876
Not sure              3348
Very conservative     4428
Very liberal          4898
dtype: int64

In [40]:
df['ideo5'] = df['ideo5'].replace('Not sure', 'Moderate or Not sure')
df['ideo5'] = df['ideo5'].replace('Moderate', 'Moderate or Not sure')

In [41]:
df.groupby(['ideo5']).size()

ideo5
Conservative            12183
Liberal                 10147
Moderate or Not sure    21224
Very conservative        4428
Very liberal             4898
dtype: int64

I'm deciding to collapse the "not sure" and "moderate" categories on the grounds that someone who is undecided about their ideology has a similar ideology to someone who does not subscribe to one side or the other.

Next, I will replace the "I'm not sure" answer to the presidential vote question, and the voter registration question, with NaN.

In [42]:
df['presvote'] = df['presvote'].replace("I'm not sure", np.NaN)
df.groupby(['presvote']).size()

presvote
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election     7688
Jill Stein (Green)                  913
Other                              1136
dtype: int64

In [43]:
df.groupby(['votereg_post']).size()

votereg_post
Don't know      279
No             3735
Yes           48885
dtype: int64

In [44]:
df['votereg_post'] = df['votereg_post'].replace("Don't know", np.NaN)
df.groupby(['votereg_post']).size()

votereg_post
No      3735
Yes    48885
dtype: int64

I do the same thing with the sexuality and trans columns:

In [45]:
df.groupby(['sexuality']).size()

sexuality
Bisexual                    1608
Gay man                     1493
Heterosexual / straight    47571
Lesbian / gay woman          637
Other                        485
Prefer not to say           1074
dtype: int64

In [46]:
df['sexuality'] = df['sexuality'].replace('Prefer not to say', np.NaN)
df.groupby(['sexuality']).size()

sexuality
Bisexual                    1608
Gay man                     1493
Heterosexual / straight    47571
Lesbian / gay woman          637
Other                        485
dtype: int64

In [47]:
df.groupby(['trans']).size()

trans
No                   51319
Prefer not to say      649
Yes                    869
dtype: int64

In [48]:
df['trans'] = df['trans'].replace('Prefer not to say', np.NaN)
df.groupby(['trans']).size()

trans
No     51319
Yes      869
dtype: int64

In [49]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse = True)

for i in range(0, 10):
    print(nan_counts[i])

('sexuality', 1105)
('trans', 711)
('gaymarriage', 431)
('banassault', 355)
('gunregistry', 325)
('concealedcarry', 311)
('unionhh', 306)
('votereg_post', 279)
('presvote', 279)
('medicarereform', 232)


In [50]:
cleaned_df = df.dropna(axis = 0, how = 'any')
len(cleaned_df.index)

46837

After dropping missing values, there are now 46823 rows remaining, out of 52899 total respondents who responded to the post-survey. This seems like a large number of points being lost. Because a large majority of respondents reported being heterosexual, and a very large majority report not being trans, I will fill in the missing values with the mode for these columns.

In [51]:
df['sexuality'] = df['sexuality'].fillna('Heterosexual / straight')
df.groupby(['sexuality']).size()

sexuality
Bisexual                    1608
Gay man                     1493
Heterosexual / straight    48676
Lesbian / gay woman          637
Other                        485
dtype: int64

In [52]:
df['trans'] = df['trans'].fillna('No')

df.groupby(['trans']).size()

trans
No     52030
Yes      869
dtype: int64

### Transforming values

Now there are a lot of variables with text values that need to be converted to binary or numeric variables.

In [53]:
df['age'] = 2018 - df['birthyr']
df = df.drop(['birthyr'], axis = 1)

Family income is a text variable indicating which range the respondent belongs to:

In [54]:
df.groupby(['faminc']).size()

faminc
$10,000 - $19,999      3543
$100,000 - $119,999    3442
$120,000 - $149,999    3021
$150,000 - $199,999    1642
$150,000 or more        136
$20,000 - $29,999      5074
$200,000 - $249,999     721
$250,000 - $349,999     380
$30,000 - $39,999      5361
$350,000 - $499,999     162
$40,000 - $49,999      4742
$50,000 - $59,999      4745
$500,000 or more        122
$60,000 - $69,999      3798
$70,000 - $79,999      3835
$80,000 - $99,999      4760
Less than $10,000      1911
Prefer not to say      5492
dtype: int64

In [55]:
df['faminc'].isnull().sum()

12

To convert family income, I will use the midpoint of the range, when a range is given. For the open ended (150,000 or more, 500,000 or more), I will use the bottom of that range. I am not sure why there was a 150,000 or more option when that overlaps with the other answers.

In [56]:
def faminc_to_int(s):
    #print(s)
    if pd.isnull(s):
        return np.NaN
    if "Prefer" in s:
        return np.NaN
    elif "Less" in s:
        return 5000
    elif "more" in s:
        incomes = int(s.replace('$', '').replace(',', '').split(' ')[0])
    else:
        incomes = [int(x) for x in s.replace('$', '').replace(' ', '').replace(',', '').split('-')]
    return int(np.mean(incomes))

faminc_to_int('$10,000 - $19,999')

14999

In [57]:
print(faminc_to_int('Less than $10,000'))
print(faminc_to_int('$500,000 or more'))

5000
500000


In [58]:
df['faminc_int'] = df['faminc'].apply(faminc_to_int)

In [59]:
df[['faminc', 'faminc_int']][0:10]

,faminc,faminc_int
0,Prefer not to say,NaN
1,"$50,000 - $59,999",54999.0
4,"$60,000 - $69,999",64999.0
5,"Less than $10,000",5000.0
6,"$20,000 - $29,999",24999.0
8,"$30,000 - $39,999",34999.0
9,"$60,000 - $69,999",64999.0
10,"$100,000 - $119,999",109999.0
11,"$40,000 - $49,999",44999.0
12,"$10,000 - $19,999",14999.0


In [60]:
df['faminc'] = df['faminc_int']
df.drop(['faminc_int'], axis = 1, inplace = True)

In [61]:
df.describe()

,faminc,Obama,Romney,age
count,47395.000000,52899.000000,52899.000000,52899.000000
mean,69937.330394,0.442258,0.315961,52.684285
std,57994.244039,0.496659,0.464902,16.157814
min,5000.000000,0.000000,0.000000,20.000000
25%,34999.000000,0.000000,0.000000,39.000000
50%,54999.000000,0.000000,0.000000,55.000000
75%,89999.000000,1.000000,1.000000,65.000000
max,500000.000000,1.000000,1.000000,97.000000


median of family income is 54999, I'll fill in the missing values accordingly.

In [62]:
df['faminc'] = df['faminc'].fillna(54999)

Now, look at which values exist in the remaining columns:

In [63]:
for col in df.columns:
    print (col, df[col].unique())

tookpost ['Yes']
gender ['Female' 'Male']
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
legalstatus ['No' 'Yes']
borderpatrol ['Yes' 'No']
legalstatusHS ['No' 'Yes']
deport ['Yes' 'No']
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 ['No' 'Yes']
milstat_2 ['No' 'Yes']
milstat_3 ['No' 'Yes']
milstat_4 ['Yes' 'No']
minwage12 ['For' 'Against' nan]
faminc [  54999.   64999.    5000.   24999.   34999.  109999.   44999.   14999.
   74999.   89999.  134999.  500000.  150000.  174999.  424999.  224999.
  299999.]
mandatorymin ['Support' 'Oppose' nan]
bodycamera ['Support' 'Oppose' nan]
pew_religimp ['Somewhat important' nan 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very libe

Many binary variables have text answers, namely support/oppose, for/against, and yes/no. I convert these to 1 and 0.

In [64]:
df1 = df.replace({'Support': 1, 'Oppose' : 0, 'For': 1, 'Against': 0, 'Yes': 1, 'No': 0, 'Favor': 1})

In [65]:
for col in df1.columns:
    print (col, df[col].unique())
    print (col, df1[col].unique())

tookpost ['Yes']
tookpost [1]
gender ['Female' 'Male']
gender ['Female' 'Male']
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
legalstatus ['No' 'Yes']
legalstatus [0 1]
borderpatrol ['Yes' 'No']
borderpatrol [1 0]
legalstatusHS ['No' 'Yes']
legalstatusHS [0 1]
deport ['Yes' 'No']
deport [1 0]
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 ['No' 'Yes']
milstat_1 [0 1]
milstat_2 ['No' 'Yes']
milstat_2 [0 1]
milstat_3 ['No' 'Yes']
milstat_3 [0 1]
milstat_4 ['Y

In [66]:
df = df1

For the five-point categorical answers, I will be converting these to dummy variables, so there is no need to do a value replacement. However, I will collapse the five answers into three answers to reduce the number of columns that are created. (may revisit this)

In [67]:
df = df.replace({'Greatly decrease': 'Decrease', 'Slightly decrease' : 'Decrease' , 'Slightly increase': 'Increase', 
                  'Greatly increase': 'Increase', 
                  'Strongly disagree': 'Disagree', 'Somewhat disagree': 'Disagree', 'Neither agree nor disagree': 'Neutral',
                 'Somewhat agree': 'Agree', 'Strongly agree': 'Agree'})

In [68]:
#list of questions with five categorical answers
fivepoint = ['whiteadvantage', 'angryracism', 'racismrare', 'fearrace', 'statewelfare', 
             'statehealthcare', 'stateedu', 'stateinfra', 'stateLE']

In [69]:
for col in fivepoint:
    df = df.merge(pd.get_dummies(df[col], prefix = col, drop_first = True), left_index = True, right_index = True)

Converting a couple more text values to 1 and 0:

In [70]:
df = df.replace({'Female': 0, 'Male' : 1})
df = df.replace({"No, didn\'t vote in a primary or caucus": 0, 'Yes, voted in a primary or caucus': 1})

In [71]:
for col in df.columns:
    print (col, df[col].unique())

tookpost [1]
gender [0 1]
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
legalstatus [0 1]
borderpatrol [1 0]
legalstatusHS [0 1]
deport [1 0]
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 [0 1]
milstat_2 [0 1]
milstat_3 [0 1]
milstat_4 [1 0]
minwage12 [  1.   0.  nan]
faminc [  54999.   64999.    5000.   24999.   34999.  109999.   44999.   14999.
   74999.   89999.  134999.  500000.  150000.  174999.  424999.  224999.
  299999.]
mandatorymin [  1.   0.  nan]
bodycamera [  1.   0.  nan]
pew_religimp ['Somewhat important' nan 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal' nan]
threestrikes [  1.   0.  nan]
pew_bornagain [  0.  nan   1.]
increasepolice [  0.  

Create binary variables based on whether the respondent has served in the military, and whether a family member has served.

In [72]:
df['veteran'] = ((df['milstat_1'] == 1) | (df['milstat_3'] == 1)).astype(int)
df['family_vet'] = ((df['milstat_2'] == 1) | (df['milstat_4'] == 1)).astype(int)

Create dummies for race:

In [73]:
df.groupby(['race']).size()

race
Asian               1739
Black               4931
Hispanic            3742
Middle Eastern        99
Mixed               1169
Native American      440
Other                680
White              40099
dtype: int64

In [74]:
df = df.merge(pd.get_dummies(df['race'], prefix = 'race', drop_first = True), left_index = True, right_index = True)

There is already a Hispanic variable (note that Hispanic isn't mutually exclusive with other races, so the two columns have different values). So I'll drop the dummy created for Hispanic.

In [75]:
df.drop(['race_Hispanic'], axis = 1, inplace = True)

Binary variable for 4-year college education or above.

In [76]:
df['college'] = ((df['educ'] == '4-year') | (df['educ'] == 'Post-grad')).astype(int)
df[['college', 'educ']].head(10)

,college,educ
0,0,High school graduate
1,0,High school graduate
4,1,4-year
5,0,High school graduate
6,0,High school graduate
8,0,High school graduate
9,0,High school graduate
10,1,4-year
11,0,Some college
12,0,Some college


Dummies for party ID:

In [77]:
df.groupby(['partyID']).size()

partyID
Democrat       20270
Independent    15787
Other           2595
Republican     14245
dtype: int64

In [78]:
df['partyID'] = df['partyID'].replace({'Other': 'Independent'})

In [79]:
df = df.merge(pd.get_dummies(df['partyID'], prefix = 'Party', drop_first = True), left_index = True, right_index = True)

In [80]:
df[['partyID', 'Party_Independent', 'Party_Republican']][:10]

,partyID,Party_Independent,Party_Republican
0,Republican,0,1
1,Independent,1,0
4,Democrat,0,0
5,Democrat,0,0
6,Republican,0,1
8,Republican,0,1
9,Republican,0,1
10,Democrat,0,0
11,Independent,1,0
12,Independent,1,0


Dummies for ideology:

In [81]:
df = df.merge(pd.get_dummies(df['ideo5'], prefix = 'ideo', drop_first = True), left_index = True, right_index = True)

In [82]:
[col for col in df.columns if "ideo" in col]

['ideo5',
 'ideo_Liberal',
 'ideo_Moderate or Not sure',
 'ideo_Very conservative',
 'ideo_Very liberal']

In [83]:
df[['ideo5', 'ideo_Liberal', 'ideo_Moderate or Not sure']][:10]

,ideo5,ideo_Liberal,ideo_Moderate or Not sure
0,Moderate or Not sure,0,1
1,Moderate or Not sure,0,1
4,Liberal,1,0
5,Moderate or Not sure,0,1
6,Very conservative,0,0
8,Conservative,0,0
9,Moderate or Not sure,0,1
10,Moderate or Not sure,0,1
11,Conservative,0,0
12,Moderate or Not sure,0,1


Dummies for religion:

In [84]:
df = df.merge(pd.get_dummies(df['religpew'], prefix = 'relig', drop_first = True), left_index = True, right_index = True)

Convert sexuality to a binary variable- straight or not straight

In [85]:
df['straight'] = (df['sexuality'] == 'Heterosexual / straight').astype(int)

Convert immstat to a binary variable indicating whether the respondent is an immigrant:

In [86]:
df.groupby(['immstat']).size()

immstat
First generation          4811
Immigrant Citizen         3121
Immigrant non-citizen      849
Second generation        11321
Third generation         32670
dtype: int64

In [87]:
df['immigrant'] = ((df['immstat'] == 'Immigrant Citizen') | (df['immstat'] == 'Immigrant non-citizen')).astype(int)

df[['immigrant', 'immstat']][:20]

,immigrant,immstat
0,0,Third generation
1,0,Third generation
4,0,Third generation
5,0,Third generation
6,0,Third generation
8,0,Third generation
9,0,Third generation
10,0,Second generation
11,0,Third generation
12,0,First generation


Convert union to a binary variable indicating whether the respondent is a union member or former union member:

In [88]:
df.groupby(['union']).size()

union
I am not now, nor have I been, a member of a labor union    38907
I formerly was a member of a labor union                    10174
Yes, I am currently a member of a labor union                3731
dtype: int64

In [89]:
df['union_member'] = ((df['union'] == 'Yes, I am currently a member of a labor union') 
                      | (df['union'] == 'I formerly was a member of a labor union')).astype(int)

df[['union_member', 'union']][5:25]

,union_member,union
8,0,"I am not now, nor have I been, a member of a l..."
9,0,"I am not now, nor have I been, a member of a l..."
10,1,I formerly was a member of a labor union
11,0,"I am not now, nor have I been, a member of a l..."
12,0,"I am not now, nor have I been, a member of a l..."
14,0,"I am not now, nor have I been, a member of a l..."
15,0,"I am not now, nor have I been, a member of a l..."
16,0,"I am not now, nor have I been, a member of a l..."
17,0,"I am not now, nor have I been, a member of a l..."
18,0,"I am not now, nor have I been, a member of a l..."


In [90]:
for col in df.columns:
    print (col, df[col].unique())

tookpost [1]
gender [0 1]
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
legalstatus [0 1]
borderpatrol [1 0]
legalstatusHS [0 1]
deport [1 0]
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 [0 1]
milstat_2 [0 1]
milstat_3 [0 1]
milstat_4 [1 0]
minwage12 [  1.   0.  nan]
faminc [  54999.   64999.    5000.   24999.   34999.  109999.   44999.   14999.
   74999.   89999.  134999.  500000.  150000.  174999.  424999.  224999.
  299999.]
mandatorymin [  1.   0.  nan]
bodycamera [  1.   0.  nan]
pew_religimp ['Somewhat important' nan 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal' nan]
threestrikes [  1.   0.  nan]
pew_bornagain [  0.  nan   1.]
increasepolice [  0.  

In [91]:
df['presvote'] = df['presvote'].replace({'Donald Trump (Republican)': 'Trump', 'Hillary Clinton (Democrat)': 'Clinton', 
                 "I didn't vote in this election": 'None', 'Gary Johnson (Libertarian)': 'Johnson',
                'Evan McMullin (Independent)': 'McMullin', 'Jill Stein (Green)': 'Stein', })

In [92]:
df = df.merge(pd.get_dummies(df['presvote'], prefix = 'pres', drop_first = True), left_index = True, right_index = True)

In [93]:
for col in df.columns:
    print(col)

tookpost
gender
educ
race
legalstatus
borderpatrol
legalstatusHS
deport
employ
milstat_1
milstat_2
milstat_3
milstat_4
minwage12
faminc
mandatorymin
bodycamera
pew_religimp
ideo5
threestrikes
pew_bornagain
increasepolice
cleanair
marstat
fuelefficiency
EPACO2
abortionchoice
abortioncoverage
federalabortion
renewables
religpew
ownhome
abortion20wks
repealACA
banmostabortion
child18
TPP
NCLB
union
primary
immstat
Iransanctions
infraspending
backgroundcheck
medicarereform
concealedcarry
gunregistry
unionhh
banassault
gaymarriage
hispanic
sexuality
investor
trans
inputstate_post
votereg_post
militaryoil
militaryterror
militarycivilwar
militarydemocracy
militaryally
militaryUN
polmeeting
polsign
campaignwork
campaigndonate
donateblood
partyID
whiteadvantage
angryracism
edloan
racismrare
fearrace
statewelfare
statehealthcare
stateedu
stateinfra
runoffice
stateLE
presvote
Obama
Romney
age
whiteadvantage_Disagree
whiteadvantage_Neutral
angryracism_Disagree
angryracism_Neutral
racismrare_Disagr

In [94]:
df.groupby(['employ']).size()

employ
Full-time               22367
Homemaker                4012
Other                     999
Part-time                5407
Permanently disabled     3323
Retired                 12236
Student                  1689
Temporarily laid off      304
Unemployed               2562
dtype: int64

In [95]:
df = df.merge(pd.get_dummies(df['employ'], prefix = 'employ', drop_first = True), left_index = True, right_index = True)

In [96]:
df.groupby(['marstat']).size()

marstat
Divorced                 5958
Domestic partnership     2211
Married                 29708
Separated                 787
Single                  11482
Widowed                  2714
dtype: int64

In [97]:
df = df.merge(pd.get_dummies(df['marstat'], prefix = 'mar', drop_first = True), left_index = True, right_index = True)

In [98]:
df.groupby(['pew_religimp']).size()

pew_religimp
Not at all important    11199
Not too important        8216
Somewhat important      13956
Very important          19504
dtype: int64

In [99]:
df = df.merge(pd.get_dummies(df['pew_religimp'], prefix = 'religimp', drop_first = True), left_index = True, right_index = True)

In [100]:
len(df.columns)

152

In [101]:
for col in df.columns:
    print (col, df[col].unique())

tookpost [1]
gender [0 1]
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
legalstatus [0 1]
borderpatrol [1 0]
legalstatusHS [0 1]
deport [1 0]
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 [0 1]
milstat_2 [0 1]
milstat_3 [0 1]
milstat_4 [1 0]
minwage12 [  1.   0.  nan]
faminc [  54999.   64999.    5000.   24999.   34999.  109999.   44999.   14999.
   74999.   89999.  134999.  500000.  150000.  174999.  424999.  224999.
  299999.]
mandatorymin [  1.   0.  nan]
bodycamera [  1.   0.  nan]
pew_religimp ['Somewhat important' nan 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal' nan]
threestrikes [  1.   0.  nan]
pew_bornagain [  0.  nan   1.]
increasepolice [  0.  

In [102]:
df = df.dropna(axis = 0, how = 'any')

df.to_csv('EDA.csv')
len(df.index)

48144

**Transforming state variables**

Creating dummies out of all 50 states would probably create too many columns. I can encode them in 6 columns using the binary representation of each state number, at the cost of interpretability.

In [103]:
states = []

for state in df['inputstate_post'].unique():
    states.append(state)
    
states = sorted(states)

state_dict = {}
for i, state in enumerate(states):
    state_dict[state] = i
    
state_dict

{'Alabama': 0,
 'Alaska': 1,
 'Arizona': 2,
 'Arkansas': 3,
 'California': 4,
 'Colorado': 5,
 'Connecticut': 6,
 'Delaware': 7,
 'District of Columbia': 8,
 'Florida': 9,
 'Georgia': 10,
 'Hawaii': 11,
 'Idaho': 12,
 'Illinois': 13,
 'Indiana': 14,
 'Iowa': 15,
 'Kansas': 16,
 'Kentucky': 17,
 'Louisiana': 18,
 'Maine': 19,
 'Maryland': 20,
 'Massachusetts': 21,
 'Michigan': 22,
 'Minnesota': 23,
 'Mississippi': 24,
 'Missouri': 25,
 'Montana': 26,
 'Nebraska': 27,
 'Nevada': 28,
 'New Hampshire': 29,
 'New Jersey': 30,
 'New Mexico': 31,
 'New York': 32,
 'North Carolina': 33,
 'North Dakota': 34,
 'Ohio': 35,
 'Oklahoma': 36,
 'Oregon': 37,
 'Pennsylvania': 38,
 'Rhode Island': 39,
 'South Carolina': 40,
 'South Dakota': 41,
 'Tennessee': 42,
 'Texas': 43,
 'Utah': 44,
 'Vermont': 45,
 'Virginia': 46,
 'Washington': 47,
 'West Virginia': 48,
 'Wisconsin': 49,
 'Wyoming': 50}

In [104]:
#function to convert a number into a six-digit binary representation, and return the digit at the specified index
def binarize(state, digit):
    num = state_dict[state]
    bin_str = format(num, '06b')
    return int(bin_str[digit])

In [105]:
df['state0'] = df['inputstate_post'].apply(lambda x: binarize(x, 0))
df['state1'] = df['inputstate_post'].apply(lambda x: binarize(x, 1))
df['state2'] = df['inputstate_post'].apply(lambda x: binarize(x, 2))
df['state3'] = df['inputstate_post'].apply(lambda x: binarize(x, 3))
df['state4'] = df['inputstate_post'].apply(lambda x: binarize(x, 4))
df['state5'] = df['inputstate_post'].apply(lambda x: binarize(x, 5))

In [106]:
df[['state0', 'state1', 'state2', 'state3', 'state4', 'state5', 'inputstate_post']].head(10)

,state0,state1,state2,state3,state4,state5,inputstate_post
0,0,1,1,1,0,1,New Hampshire
4,0,0,0,1,0,1,Colorado
5,0,0,0,0,0,0,Alabama
6,1,0,1,0,1,1,Texas
8,0,0,1,0,1,0,Georgia
9,1,0,0,1,1,0,Pennsylvania
11,1,0,1,0,1,1,Texas
12,0,0,1,0,0,1,Florida
14,0,1,0,0,0,0,Kansas
15,1,0,0,1,1,0,Pennsylvania


**Dropping columns**

For machine learning, the text variables need to be removed.

In [107]:
todrop = []
for col in df.columns:
    has_string = False
    for value in df[col].unique():
        if isinstance(value, str):
            has_string = True
    if has_string:
        todrop.append(col)
        
for col in todrop:
    print (col, df[col].unique())
#todrop = ['tookpost', 'educ', 'race', 'employ', 'pew_religimp', 'ideo5', 'marstat', 'religpew', 'ownhome', 'union', 'immstat',
#         '']

educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Other' 'Hispanic' 'Native American' 'Black' 'Asian'
 'Middle Eastern']
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Unemployed' 'Retired' 'Other' 'Student']
pew_religimp ['Somewhat important' 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal']
marstat ['Married' 'Widowed' 'Separated' 'Single' 'Divorced' 'Domestic partnership']
religpew ['Nothing in particular' 'Agnostic' 'Protestant' 'Roman Catholic' 'Atheist'
 'Something else' 'Eastern or Greek Orthodox' 'Jewish' 'Buddhist' 'Muslim'
 'Mormon' 'Hindu']
ownhome ['Own' 'Rent' 'Other']
union ['I am not now, nor have I been, a member of a labor union'
 'Yes, I am currently a member of a labor union'
 'I formerly was a member of a labor union']
immstat ['Third generation' 'First generation

In [108]:
todrop += ['milstat_1', 'milstat_2', 'milstat_3', 'milstat_4', 'tookpost']

In [109]:
df = df.drop(todrop, axis = 1)

In [110]:
len(df.columns)

129

In [111]:
df.to_csv('cleaned.csv')